## ELO init

In [ ]:
min_elo = 2550

## Filter elo

In [ ]:
# Filtering
import pandas as pd

games = pd.read_csv('./data/games_filtered_1.csv')

games = games[
    (games['winner'] == 'white') & (games['white_rating'] > min_elo) |
    (games['winner'] == 'black') & (games['black_rating'] > min_elo)
    ]
games = games.reset_index(drop=True)
games = games[['winner', 'white_rating', 'black_rating', 'moves']]
print('length:', len(games))
print(games)

games.to_csv("./data/games_"+str(min_elo)+".csv", index=False)

## Load Filtered data

In [ ]:
import pandas as pd

games = pd.read_csv('./data/games_'+str(min_elo)+'.csv')
print(games)

## Set up dataframe

In [ ]:
# columns titles
positions = ['p'+str(i) for i in range(64)]
columns = positions
columns += ['player']
columns += ['alg']
columns += ['uci']
print(columns)

## Final filtering

In [ ]:
from chess_transformation import *

games_transformed = pd.DataFrame(columns=columns)

index = 0
for i in range(len(games)):
# for i in range(1):
    print(str(i)+'/'+str(len(games)), end='\r')
    boards = []
    move_list = []
    game = games.iloc[i]

    moves = game['moves']
    winner = game['winner']
    # print(moves)
    # print(winner)

    data = algebraic_game_to_training_dataset(moves, winner)

    for i in range(len(data['boards'])):
        board = data['boards'][i]
        alg = data['moves_alg'][i]
        uci = data['moves_uci'][i]
        games_transformed.loc[index] = board+[winner]+[alg]+[uci]
        index+=1
        if index%100 == 0: # save every 100 games
            games_transformed.to_csv('./data/games_cleaned_'+str(min_elo)+'.csv')

games_transformed.to_csv('./data/games_cleaned_'+str(min_elo)+'.csv', index=False)
print(games_transformed)

## Scrambling

In [ ]:
games_cleaned = pd.read_csv('./data/games_cleaned_'+str(min_elo)+'.csv')
games_cleaned = games_cleaned.sample(frac = 1).reset_index(drop=True)
print(games_cleaned)
games_cleaned.to_csv('./data/games_cleaned_'+str(min_elo)+'_scrambled.csv', index=False)


## Turning the data to tensors

In [ ]:
import torch
from model import one_hot_encode_labels, encode_player_col
from chess_transformation import encode_uci
import pandas as pd

data_pandas = pd.read_csv('./data/games_cleaned_'+str(min_elo)+'_scrambled.csv')

position_columns = ['p'+str(i) for i in range(64)]

X_pos_pandas = data_pandas[position_columns]
X_col_pandas = data_pandas['player']

X_pos_values = X_pos_pandas.values
X_col_values = X_col_pandas.values
X_list = []
i = 0
for i in range(len(X_pos_values)):
    x_pos = X_pos_values[i]
    x_pos_encoded = one_hot_encode_labels(x_pos)
    x_col = X_col_values[i]
    x_col_encoded = encode_player_col(x_col)

    x = torch.cat((x_pos_encoded, x_col_encoded.unsqueeze(1)), dim=1)
    X_list.append(x)

X = torch.cat(X_list, dim=0)
print(X.shape)
torch.save(X, './data/X_tensor_'+str(min_elo)+'.pt')


Y_pandas = data_pandas['uci']
Y_pandas_values = Y_pandas.values
Y_list = []
for i in range(len(Y_pandas_values)):
    y_val = Y_pandas_values[i]
    y = encode_uci(y_val)
    Y_list.append(y)

Y = torch.cat(Y_list, dim=0)
print(Y.shape)
torch.save(Y, './data/Y_tensor_'+str(min_elo)+'.pt')